In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
odds = pd.read_excel('C:\\Users\\Daniel\\Documents\\Flatiron\\Capstone\\nba_odds.xlsx')
odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1228,2020,Atlanta,Detroit,10.5,NaN,NaN,NaN,NaN,NaN,NaN
1,102,2020,Atlanta,Cleveland,5.5,NaN,NaN,NaN,NaN,NaN,NaN
2,104,2020,Atlanta,NewYork,7,NaN,NaN,NaN,NaN,NaN,NaN
3,106,2020,Atlanta,Charlotte,6,NaN,NaN,NaN,NaN,NaN,NaN
4,111,2020,Atlanta,Philadelphia,6,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
odds.drop(columns=odds.columns[5:], inplace=True)

In [4]:
odds.head()

,Date,Season,Home,Away,Home Win By (Vegas)
0,1228,2020,Atlanta,Detroit,10.5
1,102,2020,Atlanta,Cleveland,5.5
2,104,2020,Atlanta,NewYork,7
3,106,2020,Atlanta,Charlotte,6
4,111,2020,Atlanta,Philadelphia,6


In [5]:
#split date column into month and day
days = []
months = []
for i in range(len(odds)):
    day = str(odds['Date'][i])[-2:]
    month = str(odds['Date'][i])[:-2]
    days.append(day)
    months.append(month)

odds['day'] = days
odds['month'] = months

odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),day,month
0,1228,2020,Atlanta,Detroit,10.5,28,12
1,102,2020,Atlanta,Cleveland,5.5,02,1
2,104,2020,Atlanta,NewYork,7,04,1
3,106,2020,Atlanta,Charlotte,6,06,1
4,111,2020,Atlanta,Philadelphia,6,11,1


In [6]:
#derive the year based on day/month/season
years = []
for i in range(len(odds)):
    if odds['Date'][i] < 1012:
        year = odds['Season'][i]+1
    else:
        year = odds['Season'][i]
    
    years.append(year)

odds['year'] = years

odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),day,month,year
0,1228,2020,Atlanta,Detroit,10.5,28,12,2020
1,102,2020,Atlanta,Cleveland,5.5,02,1,2021
2,104,2020,Atlanta,NewYork,7,04,1,2021
3,106,2020,Atlanta,Charlotte,6,06,1,2021
4,111,2020,Atlanta,Philadelphia,6,11,1,2021


In [7]:
#aggregate date column
dates = []
for i in range(len(odds)):
    date = pd.to_datetime('{}-{}-{}'.format(odds['month'][i], odds['day'][i], odds['year'][i]), format='%m-%d-%Y')
    dates.append(date)

odds['Date'] = dates
odds.head()

,Date,Season,Home,Away,Home Win By (Vegas),day,month,year
0,2020-12-28,2020,Atlanta,Detroit,10.5,28,12,2020
1,2021-01-02,2020,Atlanta,Cleveland,5.5,02,1,2021
2,2021-01-04,2020,Atlanta,NewYork,7,04,1,2021
3,2021-01-06,2020,Atlanta,Charlotte,6,06,1,2021
4,2021-01-11,2020,Atlanta,Philadelphia,6,11,1,2021


In [8]:
odds.drop(columns=['day','month','year'], inplace=True)

In [9]:
#collect all the url links for games played on everyday from 2011-2021
links = []
dates = []
for date in odds['Date'].unique():
    date = pd.to_datetime(date)
    html_page = requests.get('https://www.basketball-reference.com/boxscores/?month={}&day={}&year={}'
                             .format(date.month, date.day, date.year))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    
    soups = soup.find_all('p', {'class': 'links'})
    for i in range(len(soups)):
        entry = soups[i]
        link = entry.find('a')['href']
        links.append(link)
        dates.append('{}-{}-{}'.format(date.year,date.month,date.day))
len(links)

11656

In [10]:
len(odds)

11656

In [11]:
len(dates)

11656

In [12]:
#this dataframe will contain the raw data scraped from the website
df = pd.DataFrame(columns=['Date','Home Team','Home Points For','Home Points Against','Home eFG%','Home FTr',
                           'Home ORB%', 'Home DRB%','Home AST%','Home STL%','Home BLK%', 'Home TOV%','Home ORtg', 'Home DRtg',
                           'Away Team','Away Points For','Away Points Against','Away eFG%','Away FTr',
                           'Away ORB%', 'Away DRB%','Away AST%','Away STL%', 'Away BLK%', 'Away TOV%','Away ORTG', 'Away DRtg'],
                  index=range(len(dates)))

In [13]:
soup_order = [2, 4, 5, 6, 8, 9, 10, 11, 13, 14]

In [14]:
#for loop to scrape all the data
for i, link in enumerate(links):
    html_page = requests.get('https://www.basketball-reference.com{}'.format(link))
    soup = BeautifulSoup(html_page.content, 'html.parser')
    strong = soup('strong')
    table = soup('table')
    df['Date'][i] = dates[i]
    df['Home Team'][i] = strong[2].getText()[1:-1]
    df['Away Team'][i] = strong[1].getText()[1:-1]
    df['Home Points For'][i] = table[int(len(table)/2)]('tr')[-1]('td')[-2].getText()
    df['Away Points For'][i] = table[0]('tr')[-1]('td')[-2].getText()
    df['Home Points Against'][i] = table[0]('tr')[-1]('td')[-2].getText()
    df['Away Points Against'][i] = table[int(len(table)/2)]('tr')[-1]('td')[-2].getText()
    for ind, x in enumerate(range(4,14)):
        df[df.columns[x]][i] = table[-1]('tr')[-1]('td')[soup_order[ind]].getText()
        df[df.columns[x+13]][i] = table[int(len(table)/2)-1]('tr')[-1]('td')[soup_order[ind]].getText()
df.head()

,Date,Home Team,Home Points For,Home Points Against,Home eFG%,Home FTr,Home ORB%,Home DRB%,Home AST%,Home STL%,...,Away eFG%,Away FTr,Away ORB%,Away DRB%,Away AST%,Away STL%,Away BLK%,Away TOV%,Away ORTG,Away DRtg
0,2020-12-28,Atlanta Hawks,128,120,.610,.390,20.5,68.5,75.0,5.1,...,.505,.245,31.5,79.5,55.8,7.2,10.0,6.9,122.7,130.8
1,2020-12-28,Brooklyn Nets,111,116,.480,.170,24.5,69.8,68.3,11.9,...,.480,.296,30.2,75.5,47.6,9.1,6.9,14.6,106.0,101.5
2,2020-12-28,Denver Nuggets,124,111,.609,.368,25.0,89.1,67.4,7.7,...,.547,.267,10.9,75.0,64.1,7.7,5.4,13.5,107.0,119.5
3,2020-12-28,Los Angeles Lakers,107,115,.529,.207,13.6,83.3,48.8,6.0,...,.554,.163,16.7,86.4,62.8,9.0,10.0,8.4,115.1,107.1
4,2020-12-28,Oklahoma City Thunder,109,110,.552,.221,16.7,78.9,52.5,9.1,...,.576,.163,21.1,83.3,55.8,9.1,12.5,14.8,110.9,109.9


In [15]:
#save stats data to csv
df.to_csv("nba_game_stats.csv")

In [16]:
#save odds data to csv
odds.to_csv("nba_game_odds.csv")